### Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор

In [1]:
import os
import time
import tensorflow as tf
import numpy as np


2023-12-30 00:56:50.663939: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 00:56:50.813508: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-30 00:56:50.814938: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-30 00:56:51.697539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data_path = r'/home/dmitriy/Downloads/ai_nlp_data/hw_9/'

In [3]:
text = open(os.path.join(data_path, r'50b79a24311e90b1f359afd24bba6fc8.txt'), 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
vocab = sorted(set(text))

In [5]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

### Train и Target

In [6]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [7]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [8]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [9]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
vocab_size = len(vocab)
embedding_dim = 128
rnn_units = 1024

In [11]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [12]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)              
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')        
        self.fc = tf.keras.layers.Dense(vocab_size)


    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        x = (x + x1)/2
        return self.fc(x)


model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [24]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),     
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),           
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [25]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


### Обучение

In [27]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8731604


In [28]:
model.compile(optimizer='adam', loss=loss)

In [29]:
checkpoint_dir = os.path.join(data_path, r'training_checkpoints')
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [30]:
EPOCHS = 20

In [31]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
44/44 [==============================] - 424s 10s/step - loss: 2.4609
Epoch 2/20
44/44 [==============================] - 427s 10s/step - loss: 2.2301
Epoch 3/20
44/44 [==============================] - 423s 10s/step - loss: 1.9966
Epoch 4/20
44/44 [==============================] - 434s 10s/step - loss: 1.7648
Epoch 5/20
44/44 [==============================] - 441s 10s/step - loss: 1.6842
Epoch 6/20
44/44 [==============================] - 447s 10s/step - loss: 1.5676
Epoch 7/20
44/44 [==============================] - 441s 10s/step - loss: 1.5058
Epoch 8/20
44/44 [==============================] - 420s 10s/step - loss: 1.4572
Epoch 9/20
44/44 [==============================] - 418s 9s/step - loss: 1.4154
Epoch 10/20
44/44 [==============================] - 447s 10s/step - loss: 1.3995
Epoch 11/20
44/44 [==============================] - 462s 10s/step - loss: 1.3625
Epoch 12/20
44/44 [==============================] - 468s 11s/step - loss: 1.3438
Epoch 13/20
44/44 [=======

### Генерация текста

In [32]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_18'

In [33]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [34]:
def generate_text_new(model, start_string, num_generate=500, temperature=0.5):
    num_generate = num_generate
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = temperature
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (start_string + ''.join(text_generated))

### Тестирование

In [35]:
text_ = generate_text_new(model, start_string=u"Мой дядя самых честных", num_generate=500, temperature=0.5)
print(text_)
print(f'Длина сгенерированного текста {len(text_)}')

Мой дядя самых честных
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
Длина сгенерированного текста 522


In [36]:
text_ = generate_text_new(model, start_string=u"Мой дядя самых честных", num_generate=500, temperature=0.75)
print(text_)
print(f'Длина сгенерированного текста {len(text_)}')

Мой дядя самых честных пизор, да зонье, небы
                                                                Яг масвиролье ручильный ща зилиз, левотоньо
                                                    VLVI

                                      Дрипимной полга возни набона.
                                                                                                                                                                                                                                                   
Длина сгенерированного текста 522


In [37]:
text_ = generate_text_new(model, start_string=u"Мой дядя самых честных", num_generate=500, temperature=1.0)
print(text_)
print(f'Длина сгенерированного текста {len(text_)}')

Мой дядя самых честных
                         Ийжи жифдив он етдолат возшый
                         Матький                            XЧ те Гамощ х вленет у талят.
                            Овона пепялай, Лауйлогайн кий,
                                           О           Матанне уте прому и татет.
                                 Татдяка жеск ялаваце старе:
                               X           Постадья унькой преч имоко.

                                                   Нрамовь дро, гебсе Трочетивий
Длина сгенерированного текста 522


In [38]:
text_ = generate_text_new(model, start_string=u"Мой дядя самых честных", num_generate=50, temperature=1.0)
print(text_)
print(f'Длина сгенерированного текста {len(text_)}')

Мой дядя самых честных
                                                 
Длина сгенерированного текста 72


In [39]:
text_ = generate_text_new(model, start_string=u"Мой дядя самых честных", num_generate=50, temperature=1.5)
print(text_)
print(f'Длина сгенерированного текста {len(text_)}')

Мой дядя самых честных вай нают.
          Стьрдолно!я;  
              
Длина сгенерированного текста 72


In [40]:
text_ = generate_text_new(model, start_string=u"Мой дядя самых честных", num_generate=25, temperature=1.1)
print(text_)
print(f'Длина сгенерированного текста {len(text_)}')

Мой дядя самых честных.
                       
Длина сгенерированного текста 47
